In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive, files
uploaded = files.upload()

Saving data_q6_q7.xlsx to data_q6_q7.xlsx


In [ ]:
def sigmoid(z):
  z = z.astype(float)
  z_out = 1/(1 + np.exp(-z))
  return z_out

In [ ]:
def hypothesis(X, wt):
  hyp = sigmoid(np.dot(X, wt.T))
  return hyp

In [ ]:
# function for normalising data
def norm(data):
  # norm_data = data
  mean = np.mean(data, axis = 0)
  std = np.std(data, axis = 0)
  norm_data = (data-mean)/std
  return norm_data

In [ ]:
# function for regularisation
def wt_regularisation(lamb, wt, reg):
  wt_reg = np.zeros(wt.shape)
  if reg == 0:
    wt_reg = 0
  elif reg == 1:
    wt_reg = (lamb/2)*np.sign(wt)
  elif reg == 2:
    wt_reg = lamb*wt
  return wt_reg

In [ ]:
def bgd(alpha, lamb, iters, X, y, reg):
  w = np.zeros(X.shape[1], dtype=np.longfloat)
  for i in range(iters):
    hyp = hypothesis(X, wt)
    w = w - (alpha/len(y))*(np.dot(hyp - y, X) - wt_regularisation(lamb, w, reg))
    # w = w*(1 - (alpha/len(y))*lamb) - (alpha/len(y))*np.dot(hyp - y, X)
  return w

In [ ]:
def mbgd(alpha, lamb, iters, batch_size, X, y, reg):
  w = np.random.rand(X.shape[1])
  for i in range(iters):
    rand_ind = np.random.randint(len(y))
    X_batch = X[rand_ind:rand_ind + batch_size]
    y_batch = y[rand_ind:rand_ind + batch_size]
    hyp = hypothesis(X_batch, wt)
    w = w - (alpha/len(y))*(np.dot(hyp-y_batch, X_batch) - wt_regularisation(lamb, w, reg))
  return w

In [ ]:
def sgd(alpha, lamb, iters, X, y, reg):
  w = np.random.rand(X.shape[1])
  for i in range(iters):
    rand_ind = np.random.randint(len(y))
    X_ind = X[rand_ind:rand_ind + 1]
    y_ind = y[rand_ind:rand_ind + 1]
    hyp = hypothesis(X_ind, wt)
    # print(hyp.shape)
    # print(y_ind.shape)
    w = w - (alpha/len(y))*(np.dot(hyp - y_ind, X_ind) - wt_regularisation(lamb, w, reg))
  return w

In [ ]:
# splitting the data into folds
def cross_validation(k, X, y):
  # print(len(X))
  fold_size = int(len(X)/5)
  # print(fold_size)
  X_ts = X[k*fold_size:(k+1)*fold_size]
  y_ts = y[k*fold_size:(k+1)*fold_size]
  X_tr = np.delete(X, slice(k*fold_size, (k+1)*fold_size), axis = 0)
  y_tr = np.delete(y, slice(k*fold_size, (k+1)*fold_size), axis = 0)
  # print(len(X_ts), len(y_ts))
  # print(len(X_tr), len(y_tr))
  return X_ts, y_ts, X_tr, y_tr

In [ ]:
def classification(X_ts, wt):
  m = X_ts.shape[0]
  y_sig = hypothesis(X_ts, wt)
  # print(y_sig)
  y_pred = np.zeros(m)
  for i in range(m):
    if y_sig[i]>=0.5:
      y_pred[i] = 1
    elif y_sig[i]<0.5:
      y_pred[i] = 0
  return y_pred

In [ ]:
def confusion_mat(y_predicted, y_testing):
  conf_mat = np.zeros((3,3))
  for i in range(len(y_testing)):
    if y_testing[i] == 1:
      if y_predicted[i] == 1:
        conf_mat [0][0] += 1
      if y_predicted[i] == 2:
        conf_mat [0][1] += 1
      if y_predicted[i] == 3:
        conf_mat [0][2] += 1
    if y_testing[i] == 2:
      if y_predicted[i] == 1:
        conf_mat [1][0] += 1
      if y_predicted[i] == 2:
        conf_mat [1][1] += 1
      if y_predicted[i] == 3:
        conf_mat [1][2] += 1
    if y_testing[i] == 3:
      if y_predicted[i] == 1:
        conf_mat [2][0] += 1
      if y_predicted[i] == 2:
        conf_mat [2][1] += 1
      if y_predicted[i] == 3:
        conf_mat [2][2] += 1
  acc1 = conf_mat[ 0 ][ 0 ]/sum(conf_mat[ 0 ])
  acc2 = conf_mat[ 1 ][ 1 ]/sum(conf_mat[ 1 ])
  acc3 = conf_mat[ 2 ][ 2 ]/sum(conf_mat[ 2 ])
  acc = (conf_mat[ 0 ][ 0 ] + conf_mat[ 1 ][ 1 ] + conf_mat[ 2 ][ 2 ])/np.sum(conf_mat)
  return acc1, acc2, acc3, acc

In [ ]:
# extracting the data and separating
data = pd.read_excel("data_q6_q7.xlsx")
data = np.asarray(data)
np.random.shuffle(data)

In [ ]:
# splitting into input and output
X = data[:, :-1]  #input
y = data[:,-1]    #output

# normalizing X and y
X = X.astype(float)
X = norm(X)

# defining X for regression model
m = X.shape[0]
one = np.ones([m,1])
X = np.append(one, X, axis = 1)

In [ ]:
def ysplit(y_tr):
  y1_tr = [ 1 for i in range(len(y_tr))]
  y2_tr = [ 1 for i in range(len(y_tr))]
  y3_tr = [ 1 for i in range(len(y_tr))]
  for i in range(len(y_tr)):
    if (y_tr[i] != 1 ):
     y1_tr[i] = 0
    if (y_tr[i] != 2 ):
      y2_tr[i] = 0
    if (y_tr[i] != 3 ):
      y3_tr[i] = 0
  return y1_tr, y2_tr, y3_tr

In [ ]:
print('ONE vs ALL')
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = bgd(a, l, 1000, X_tr, y1_tr, 0)
  yp1 = classification(X_ts, wt)
  wt = bgd(a, l, 1000, X_tr, y2_tr, 0)
  yp2 = classification(X_ts, wt)
  wt = bgd(a, l, 1000, X_tr, y3_tr, 0)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    if (yp2[i] == 1 ):
      cval[i] = 2.0
    if (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts, name= 'Actual' )
  y_pred = pd.Series(cval, name= 'Predicted' )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('BGD LOR without regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

ONE vs ALL
BGD LOR without regularisation
Average individual accuracy of class 1: 0.26302031302031303
Average individual accuracy of class 2: 1.0
Average individual accuracy of class 3: 1.0
Average overall accuracy: 0.763780487804878


In [ ]:
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = bgd(a, l, 1000, X_tr, y1_tr, 1)
  yp1 = classification(X_ts, wt)
  wt = bgd(a, l, 1000, X_tr, y2_tr, 1)
  yp2 = classification(X_ts, wt)
  wt = bgd(a, l, 1000, X_tr, y3_tr, 1)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    if (yp2[i] == 1 ):
      cval[i] = 2.0
    if (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts, name= 'Actual' )
  y_pred = pd.Series(cval, name= 'Predicted' )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('BGD LOR with L1 regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

BGD LOR with L1 regularisation
Average individual accuracy of class 1: 0.3011868686868687
Average individual accuracy of class 2: 1.0
Average individual accuracy of class 3: 1.0
Average overall accuracy: 0.7645121951219512


In [ ]:
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = bgd(a, l, 1000, X_tr, y1_tr, 2)
  yp1 = classification(X_ts, wt)
  wt = bgd(a, l, 1000, X_tr, y2_tr, 2)
  yp2 = classification(X_ts, wt)
  wt = bgd(a, l, 1000, X_tr, y3_tr, 2)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    if (yp2[i] == 1 ):
      cval[i] = 2.0
    if (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts, name= 'Actual' )
  y_pred = pd.Series(cval, name= 'Predicted' )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('BGD LOR with L2 regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

BGD LOR with L2 regularisation
Average individual accuracy of class 1: 0.26302031302031303
Average individual accuracy of class 2: 1.0
Average individual accuracy of class 3: 1.0
Average overall accuracy: 0.763780487804878


In [ ]:
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = sgd(a, l, 2000, X_tr, y1_tr, 0)
  yp1 = classification(X_ts, wt)
  wt = sgd(a, l, 1000, X_tr, y2_tr, 0)
  yp2 = classification(X_ts, wt)
  wt = sgd(a, l, 1000, X_tr, y3_tr, 0)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    if (yp2[i] == 1 ):
      cval[i] = 2.0
    if (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts, name= 'Actual' )
  y_pred = pd.Series(cval, name= 'Predicted' )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('SGD LOR without regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

SGD LOR without regularisation
Average individual accuracy of class 1: 0.025
Average individual accuracy of class 2: 0.015384615384615385
Average individual accuracy of class 3: 0.7
Average overall accuracy: 0.08838915470494417


In [ ]:
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = sgd(a, l, 2000, X_tr, y1_tr, 1)
  yp1 = classification(X_ts, wt)
  wt = sgd(a, l, 2000, X_tr, y2_tr, 1)
  yp2 = classification(X_ts, wt)
  wt = sgd(a, l, 5000, X_tr, y3_tr, 1)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    if (yp2[i] == 1 ):
      cval[i] = 2.0
    if (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts, name= 'Actual' )
  y_pred = pd.Series(cval, name= 'Predicted' )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('SGD LOR with L1 regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

SGD LOR with L1 regularisation
Average individual accuracy of class 1: 0.07222222222222222
Average individual accuracy of class 2: 0.0
Average individual accuracy of class 3: 0.6485714285714286
Average overall accuracy: 0.07192355330503998


In [ ]:
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = sgd(a, l, 2000, X_tr, y1_tr, 2)
  yp1 = classification(X_ts, wt)
  wt = sgd(a, l, 2000, X_tr, y2_tr, 2)
  yp2 = classification(X_ts, wt)
  wt = sgd(a, l, 4000, X_tr, y3_tr, 2)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    if (yp2[i] == 1 ):
      cval[i] = 2.0
    if (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts, name= 'Actual' )
  y_pred = pd.Series(cval, name= 'Predicted' )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('SGD LOR with L2 regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

SGD LOR with L2 regularisation
Average individual accuracy of class 1: 0.2
Average individual accuracy of class 2: 0.0
Average individual accuracy of class 3: 0.35
Average overall accuracy: 0.07942402425161045


In [ ]:
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = mbgd(a, l, 1000, 20, X_tr, y1_tr, 0)
  yp1 = classification(X_ts, wt)
  wt = mbgd(a, l, 1000, 20, X_tr, y2_tr, 0)
  yp2 = classification(X_ts, wt)
  wt = mbgd(a, l, 1000, 20, X_tr, y3_tr, 0)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    elif (yp2[i] == 1 ):
      cval[i] = 2.0
    elif (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts, name= 'Actual' )
  y_pred = pd.Series(cval, name= 'Predicted' )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('MBGD LOR without regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

MBGD LOR without regularisation
Average individual accuracy of class 1: 0.9527777777777778
Average individual accuracy of class 2: 0.0
Average individual accuracy of class 3: 0.0
Average overall accuracy: 0.31354212454212455


In [ ]:
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = mbgd(a, l, 1000, 20, X_tr, y1_tr, 1)
  yp1 = classification(X_ts, wt)
  wt = mbgd(a, l, 1000, 20, X_tr, y2_tr, 1)
  yp2 = classification(X_ts, wt)
  wt = mbgd(a, l, 1000, 20, X_tr, y3_tr, 1)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    elif (yp2[i] == 1 ):
      cval[i] = 2.0
    elif (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts)
  y_pred = pd.Series(cval )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('MBGD LOR with L1 regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

MBGD LOR with L1 regularisation
Average individual accuracy of class 1: 0.8466666666666665
Average individual accuracy of class 2: 0.0
Average individual accuracy of class 3: 0.2
Average overall accuracy: 0.318686106921401


In [ ]:
wt = np.random.rand(X.shape[1])
ind_acc1 = []
ind_acc2 = []
ind_acc3 = []
overall_acc = []
for i in range(5):
  X_ts, y_ts, X_tr, y_tr = cross_validation(i, X, y)
  a = 0.0005
  l = 0.0001
  y1_tr, y2_tr, y3_tr = ysplit(y_tr)
  wt = mbgd(a, l, 1000, 20, X_tr, y1_tr, 2)
  yp1 = classification(X_ts, wt)
  wt = mbgd(a, l, 1000, 20, X_tr, y2_tr, 2)
  yp2 = classification(X_ts, wt)
  wt = mbgd(a, l, 1000, 20, X_tr, y3_tr, 2)
  yp3 = classification(X_ts, wt)

  cval = [ 0 for i in range(len(y_ts))]
  for i in range(len(y_ts)):
    if (yp1[i] == 1 ):
      cval[i] = 1.0
    elif (yp2[i] == 1 ):
      cval[i] = 2.0
    elif (yp3[i] == 1 ):
      cval[i] = 3.0
  for i in range(len(cval)):
    if (cval[i] == 0 ):
      cval[i] = 'None'
  y_actual = pd.Series(y_ts)
  y_pred = pd.Series(cval )

  acc1, acc2, acc3, acc = confusion_mat(y_pred, y_ts)

  ind_acc1.append(acc1)
  ind_acc2.append(acc2)
  ind_acc3.append(acc3)
  overall_acc.append(acc)


print('MBGD LOR with L2 regularisation')
avg_ind_acc1 = sum(ind_acc1)/len(ind_acc1)
avg_ind_acc2 = sum(ind_acc2)/len(ind_acc2)
avg_ind_acc3 = sum(ind_acc3)/len(ind_acc3)
avg_overall_acc = sum(overall_acc)/len(overall_acc)
print("Average individual accuracy of class 1:", avg_ind_acc1)
print("Average individual accuracy of class 2:", avg_ind_acc2)
print("Average individual accuracy of class 3:", avg_ind_acc3)
print("Average overall accuracy:", avg_overall_acc)

MBGD LOR with L2 regularisation
Average individual accuracy of class 1: 0.6900000000000001
Average individual accuracy of class 2: 0.0
Average individual accuracy of class 3: 0.0
Average overall accuracy: 0.2178567936005007
